In [ ]:
%load_ext autoreload
%autoreload 2
from load import *
import torch
import yaml
import logging
from imp import reload
import matplotlib.pyplot as plt
from liftpose.vision_3d import world_to_camera_dict, reprojection_error
reload(logging)
logger = logging.getLogger(__name__).setLevel(logging.INFO)

# decleare data parameters
par_train = {'data_dir' : '/data/LiftPose3D/fly_tether/data_DF3D', # change the path 
               'out_dir'        : './out',
               'train_subjects' : [1],
               'test_subjects'  : [6,7],
               'actions'        : ['all'],
               'cam_id'         : [2]}

# merge with training parameters
par_data = yaml.full_load(open('param.yaml', "rb"))
par = {**par_data["data"], **par_train}

# Load 2D data
train_2d = load_2D(
    par["data_dir"],
    par,
    cam_id=par["cam_id"],
    subjects=par["train_subjects"],
    actions=par["actions"],
)
test_2d = load_2D(
    par["data_dir"],
    par,
    cam_id=par["cam_id"],
    subjects=par["test_subjects"],
    actions=par["actions"],
)

# Load 3D data
train_3d, _, rcams_train = load_3D(
    par["data_dir"],
    par,
    cam_id=par["cam_id"],
    subjects=par["train_subjects"],
    actions=par["actions"],
)
test_3d, _, rcams_test = load_3D(
    par["data_dir"],
    par,
    cam_id=par["cam_id"],
    subjects=par["test_subjects"],
    actions=par["actions"],
)

#train_3d = transform_frame(train_3d, rcams_train)
#test_3d = transform_frame(test_3d, rcams_test)

In [ ]:
dat = pickle.load(open('/home/user/Desktop/df3d_anomaly/pose_result/pose_result_fix__data_paper_180918_MDN_CsCh_Fly1_001_SG1_behData_images.pkl', 'rb'))

pts3d = dat["points3d"]
pts2d = dat["points2d"][2]
cam = dat[2]
proj = reprojection_error(pts3d, pts2d, cam['R'], cam['tvec'], cam['intr'])
(np.linalg.norm(proj - pts2d, axis=-1)[:,:15])

In [ ]:
k = list(train_3d)[1]
print(k)
cam_id = 2
k_2d = (k[0], k[1], k[2]+f'.cam_{cam_id}')
cam =  rcams_train[k][cam_id]
proj = reprojection_error(train_3d[k], train_2d[k_2d], cam['R'], cam['tvec'], cam['intr'])[:, :15] 